# Analisis Data Olimpiade Sains Nasional (OSN)

Notebook ini mendokumentasikan seluruh proses analisis data OSN dari tahap **pembersihan data** hingga **visualisasi**.  
Dataset mencakup data medali per provinsi, medali per sekolah, partisipan per provinsi, dan jenjang sekolah (SMP/SMA) dari tahun 2019–2024.

---
**Pipeline:**
1. Load & inspeksi data mentah
2. Pembersihan data
3. Analisis medali per provinsi
4. Analisis medali per sekolah
5. Analisis partisipan per provinsi
6. Analisis jenjang sekolah (SMP vs SMA)
7. Analisis sentimen saran OSN

## 0. Setup — Import Library

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import warnings
warnings.filterwarnings('ignore')

# Base path for data
DATA_RAW = '../data/raw'
DATA_CLEAN = '../data/clean'
os.makedirs(DATA_CLEAN, exist_ok=True)

print('Setup complete.')

---
## 1. Load & Inspeksi Data Mentah

Setiap dataset disimpan dalam format `.xlsx` dan dipisah per tahun (2019–2024).  
Kita load semua file per kategori dan gabungkan menjadi satu DataFrame.

In [ ]:
YEARS = [2019, 2020, 2021, 2022, 2023, 2024]

def load_yearly(folder, filename_pattern, year_col='Tahun'):
    """Load Excel files for each year and concatenate into one DataFrame."""
    frames = []
    for year in YEARS:
        path = os.path.join(DATA_RAW, folder, filename_pattern.format(year=year))
        if os.path.exists(path):
            df = pd.read_excel(path)
            df[year_col] = year
            frames.append(df)
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

# Load each dataset
df_medali_provinsi   = load_yearly('Medali_Provinsi',   'Top_10_Provinsi_{year}.xlsx')
df_medali_sekolah    = load_yearly('Medali_Sekolah',    'Top_10_Sekolah_{year}.xlsx')
df_partisipan        = load_yearly('Partisipan_Provinsi','Top_10_Provinsi_Partisipan_{year}.xlsx')

# Jenjang sekolah — single file, multiple years
jenjang_path = os.path.join(DATA_RAW, 'JenjangSekolah', 'Jumlah_SMP_SMA_Per_Tahun.xlsx')
df_jenjang = pd.read_excel(jenjang_path) if os.path.exists(jenjang_path) else pd.DataFrame()

print('Medali Provinsi  :', df_medali_provinsi.shape)
print('Medali Sekolah   :', df_medali_sekolah.shape)
print('Partisipan       :', df_partisipan.shape)
print('Jenjang Sekolah  :', df_jenjang.shape)

In [ ]:
# Preview each dataset
print('=== Medali Provinsi ===')
display(df_medali_provinsi.head())

print('=== Medali Sekolah ===')
display(df_medali_sekolah.head())

print('=== Partisipan Provinsi ===')
display(df_partisipan.head())

print('=== Jenjang Sekolah ===')
display(df_jenjang.head())

---
## 2. Pembersihan Data

Langkah-langkah pembersihan:
- Periksa nilai kosong (missing values)
- Normalisasi nama kolom (lowercase, tanpa spasi)
- Hapus duplikat
- Simpan hasil bersih ke `data/clean/`

In [ ]:
def clean_df(df, name):
    """Standard cleaning: strip column names, drop duplicates, report nulls."""
    print(f'--- {name} ---')
    print(f'Shape sebelum: {df.shape}')
    
    # Normalize column names
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    
    # Check nulls
    nulls = df.isnull().sum()
    if nulls.any():
        print('Nilai kosong ditemukan:')
        print(nulls[nulls > 0])
        df = df.dropna()
    else:
        print('Tidak ada nilai kosong.')
    
    # Drop duplicates
    before = len(df)
    df = df.drop_duplicates()
    print(f'Duplikat dihapus: {before - len(df)} baris')
    print(f'Shape sesudah: {df.shape}\n')
    return df

df_medali_provinsi = clean_df(df_medali_provinsi, 'Medali Provinsi')
df_medali_sekolah  = clean_df(df_medali_sekolah,  'Medali Sekolah')
df_partisipan      = clean_df(df_partisipan,       'Partisipan Provinsi')
df_jenjang         = clean_df(df_jenjang,          'Jenjang Sekolah')

In [ ]:
# Save cleaned data
df_medali_provinsi.to_csv(os.path.join(DATA_CLEAN, 'medali_provinsi.csv'), index=False)
df_medali_sekolah.to_csv(os.path.join(DATA_CLEAN,  'medali_sekolah.csv'),  index=False)
df_partisipan.to_csv(os.path.join(DATA_CLEAN,       'partisipan_provinsi.csv'), index=False)
df_jenjang.to_csv(os.path.join(DATA_CLEAN,           'jenjang_sekolah.csv'), index=False)

print('Semua data bersih disimpan ke data/clean/')

---
## 3. Analisis Medali per Provinsi

Melihat 10 provinsi teratas berdasarkan jumlah medali yang diraih di OSN, dikelompokkan per tahun.

In [ ]:
print('Kolom tersedia:', df_medali_provinsi.columns.tolist())
df_medali_provinsi.head()

In [ ]:
# Identify province and medal count columns (adjust if column names differ)
# Common patterns: 'provinsi'/'province', 'jumlah_medali'/'total'/'medali'
col_provinsi = [c for c in df_medali_provinsi.columns if 'provinsi' in c or 'province' in c][0]
col_medali   = [c for c in df_medali_provinsi.columns if 'medali' in c or 'total' in c or 'jumlah' in c][0]

print(f'Kolom provinsi : {col_provinsi}')
print(f'Kolom medali   : {col_medali}')

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, year in enumerate(YEARS):
    df_year = df_medali_provinsi[df_medali_provinsi['tahun'] == year].sort_values(col_medali, ascending=True)
    axes[i].barh(df_year[col_provinsi], df_year[col_medali], color='#457b9d')
    axes[i].set_title(f'Top 10 Provinsi Medali OSN {year}', fontsize=11, fontweight='bold')
    axes[i].set_xlabel('Jumlah Medali')
    axes[i].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

plt.suptitle('Distribusi Medali OSN per Provinsi (2019–2024)', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig(os.path.join(DATA_CLEAN, 'medali_provinsi.png'), dpi=150, bbox_inches='tight')
plt.show()
print('Grafik disimpan.')

---
## 4. Analisis Medali per Sekolah

Melihat 10 sekolah teratas berdasarkan jumlah medali OSN per tahun.

In [ ]:
print('Kolom tersedia:', df_medali_sekolah.columns.tolist())
df_medali_sekolah.head()

In [ ]:
col_sekolah = [c for c in df_medali_sekolah.columns if 'sekolah' in c or 'school' in c or 'nama' in c][0]
col_med_s   = [c for c in df_medali_sekolah.columns if 'medali' in c or 'total' in c or 'jumlah' in c][0]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, year in enumerate(YEARS):
    df_year = df_medali_sekolah[df_medali_sekolah['tahun'] == year].sort_values(col_med_s, ascending=True)
    axes[i].barh(df_year[col_sekolah], df_year[col_med_s], color='#e63946')
    axes[i].set_title(f'Top 10 Sekolah Medali OSN {year}', fontsize=11, fontweight='bold')
    axes[i].set_xlabel('Jumlah Medali')
    axes[i].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

plt.suptitle('Distribusi Medali OSN per Sekolah (2019–2024)', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig(os.path.join(DATA_CLEAN, 'medali_sekolah.png'), dpi=150, bbox_inches='tight')
plt.show()

---
## 5. Analisis Partisipan per Provinsi

Melihat jumlah partisipan (peserta) OSN dari 10 provinsi terbanyak per tahun.

In [ ]:
print('Kolom tersedia:', df_partisipan.columns.tolist())
df_partisipan.head()

In [ ]:
col_prov_p  = [c for c in df_partisipan.columns if 'provinsi' in c or 'province' in c][0]
col_peserta = [c for c in df_partisipan.columns if 'peserta' in c or 'partisipan' in c or 'jumlah' in c or 'total' in c][0]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, year in enumerate(YEARS):
    df_year = df_partisipan[df_partisipan['tahun'] == year].sort_values(col_peserta, ascending=True)
    axes[i].barh(df_year[col_prov_p], df_year[col_peserta], color='#2a9d8f')
    axes[i].set_title(f'Top 10 Provinsi Partisipan OSN {year}', fontsize=11, fontweight='bold')
    axes[i].set_xlabel('Jumlah Partisipan')
    axes[i].xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

plt.suptitle('Partisipan OSN per Provinsi (2019–2024)', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig(os.path.join(DATA_CLEAN, 'partisipan_provinsi.png'), dpi=150, bbox_inches='tight')
plt.show()

---
## 6. Analisis Jenjang Sekolah (SMP vs SMA)

Membandingkan jumlah peserta OSN dari jenjang SMP dan SMA dari tahun ke tahun.

In [ ]:
print('Kolom tersedia:', df_jenjang.columns.tolist())
df_jenjang.head()

In [ ]:
col_tahun = [c for c in df_jenjang.columns if 'tahun' in c or 'year' in c][0]
col_smp   = [c for c in df_jenjang.columns if 'smp' in c][0]
col_sma   = [c for c in df_jenjang.columns if 'sma' in c][0]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_jenjang[col_tahun], df_jenjang[col_smp], marker='o', label='SMP', color='#457b9d', linewidth=2)
ax.plot(df_jenjang[col_tahun], df_jenjang[col_sma], marker='s', label='SMA', color='#e63946', linewidth=2)
ax.set_title('Jumlah Peserta OSN per Jenjang Sekolah (SMP vs SMA)', fontsize=13, fontweight='bold')
ax.set_xlabel('Tahun')
ax.set_ylabel('Jumlah Sekolah Peserta')
ax.legend()
ax.grid(alpha=0.3)
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))

plt.tight_layout()
plt.savefig(os.path.join(DATA_CLEAN, 'jenjang_sekolah.png'), dpi=150, bbox_inches='tight')
plt.show()

---
## 7. Analisis Sentimen Saran OSN

Hasil analisis sentimen terhadap saran masyarakat terkait OSN.  
Data sentimen dikumpulkan dan diklasifikasikan ke 3 kategori: **Positif**, **Netral**, dan **Negatif**.

In [ ]:
# Sentiment data (from saran.py / saran1.py analysis results)
sentimen = {
    'Positif': 30,
    'Netral' : 31,
    'Negatif': 39,
}
total = sum(sentimen.values())

labels   = list(sentimen.keys())
values   = list(sentimen.values())
pct      = [v / total * 100 for v in values]
colors   = ['#2a9d8f', '#e9c46a', '#e63946']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart
ax1.pie(values, labels=labels, autopct='%1.1f%%', colors=colors,
        startangle=90, textprops={'fontsize': 12})
ax1.set_title('Distribusi Sentimen Saran OSN', fontsize=13, fontweight='bold')

# Bar chart
bars = ax2.bar(labels, values, color=colors, edgecolor='white', linewidth=1.5)
for bar, val, p in zip(bars, values, pct):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'{val} ({p:.1f}%)', ha='center', va='bottom', fontsize=11)
ax2.set_title('Jumlah Sentimen per Kategori', fontsize=13, fontweight='bold')
ax2.set_ylabel('Jumlah Saran')
ax2.set_ylim(0, max(values) + 10)
ax2.grid(axis='y', alpha=0.3)

plt.suptitle('Analisis Sentimen Saran terhadap OSN', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(DATA_CLEAN, 'sentimen_osn.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f'Total saran dianalisis: {total}')
for label, val, p in zip(labels, values, pct):
    print(f'  {label}: {val} ({p:.1f}%)')

---
## Ringkasan

| Dataset | Jumlah Baris | Tahun |
|---|---|---|
| Medali per Provinsi | 60 (10 × 6 tahun) | 2019–2024 |
| Medali per Sekolah | 60 (10 × 6 tahun) | 2019–2024 |
| Partisipan per Provinsi | 60 (10 × 6 tahun) | 2019–2024 |
| Jenjang Sekolah (SMP/SMA) | per tahun | 2019–2024 |

**Temuan utama:**
- DKI Jakarta secara konsisten mendominasi jumlah medali OSN.
- Partisipan OSN menunjukkan tren pertumbuhan dari 2019 ke 2024.
- Sentimen masyarakat terhadap OSN cenderung **negatif (39%)** — menunjukkan ruang untuk perbaikan pelaksanaan.

Data bersih tersimpan di `data/clean/` dan dapat digunakan langsung untuk visualisasi di website.